In [13]:
# 必要なモジュールをインポート
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from openai.types.chat import ChatCompletionToolParam
from tavily import TavilyClient

# 環境変数の取得
load_dotenv("../.env")

# OpenAI APIクライアントを生成
client = OpenAI(api_key=os.environ['API_KEY'])

# tavily検索用APIキーの取得
TAVILY_API_KEY = os.environ['TAVILY_API_KEY']

# モデル名
MODEL_NAME = "gpt-4o-mini"

# 検索結果を返す関数の作成
def get_search_result(question):
    print("------get_search_result(検索結果を返す関数)------")
    print(f"質問: {question}")
    # TavilyClientを使用して検索結果を取得
    client = TavilyClient(api_key=TAVILY_API_KEY)
    response = client.search(question)
    return json.dumps({"result": response["results"]})

# テスト用コード
# ret = get_search_result("東京駅のイベントを教えて")
# json.loads(ret)

# ツール定義
def define_tools():
    print("------define_tools(ツール定義)------")
    return [
        ChatCompletionToolParam({
            "type": "function",
            "function": {
                "name": "get_search_result",
                "description": "指定した質問文の検索結果を取得する",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "question": {"type": "string", "description": "質問文"},
                    },
                    "required": ["question"],
                },
            },
        })
    ]

# 言語モデルへの質問を行う関数
def ask_question(question, tools):
    print("------ask_question(言語モデルへの質問を行う関数)------")
    print(f"質問: {question}")
    print(f"使用するツール: {tools}")
    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[{"role": "user", "content": question}],
        tools=tools,
        tool_choice="auto",
    )
    return response

# ツール呼び出しが必要な場合の処理を行う関数
def handle_tool_call(response, question):
    print("------handle_tool_call(ツール呼び出しが必要な場合の処理を行う関数)------")
    print(f"レスポンス: {response}")
    print(f"質問: {question}")
    # 関数の実行と結果取得
    tool = response.choices[0].message.tool_calls[0]
    function_name = tool.function.name
    arguments = json.loads(tool.function.arguments)
    function_response = globals()[function_name](**arguments)

    # 関数の実行結果をmessagesに加えて再度言語モデルを呼出
    response_after_tool_call = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[
            {"role": "user", "content": question},
            response.choices[0].message,
            {
                "tool_call_id": tool.id,
                "role": "tool",
                "content": function_response,
            },
        ],
    )
    return response_after_tool_call

# ユーザーからの質問を処理する関数
def process_response(question, tools):
    print("------process_response(ユーザーからの質問を処理する関数)------")
    print(f"質問: {question}")
    print(f"使用するツール: {tools}")
    # 言語モデルに質問を投げる
    response = ask_question(question, tools)

    print(f"response.choices[0].finish_reasonの中身： {response.choices[0].finish_reason}")
    # レスポンスのfinish_reasonを確認
    if response.choices[0].finish_reason == 'tool_calls':
        # ツール呼出の場合
        print("------★★★★ツール呼び出し★★★★------")
        final_response = handle_tool_call(response, question)
        return final_response.choices[0].message.content.strip()
    else:
        # 言語モデルが直接回答する場合
        print("------★★★★言語モデルが直接回答★★★★------")
        return response.choices[0].message.content.strip()

# 処理開始
print("------処理開始------")
# ツールを定義
tools = define_tools()

# 言語モデルが直接回答できる質問
# case_name ="言語モデルが直接回答できる質問"
# question = "東京都と沖縄県はどちらが広いですか？"

# ツール呼出が必要な質問
case_name ="ツール呼出が必要な質問"
question = "東京駅のイベントについて、最近1ヶ月以内の検索結果を教えてください"

# case_name = "ツール呼出が必要な質問" if "ツール呼出" in question else "言語モデルが直接回答できる質問"
print(f"------★★★★ケース名: {case_name}★★★★------")

response_message = process_response(question, tools)
print("------結果の表示------")
print(response_message)

------処理開始------
------define_tools(ツール定義)------
------★★★★ケース名: ツール呼出が必要な質問★★★★------
------process_response(ユーザーからの質問を処理する関数)------
質問: 東京駅のイベントについて、最近1ヶ月以内の検索結果を教えてください
使用するツール: [{'type': 'function', 'function': {'name': 'get_search_result', 'description': '指定した質問文の検索結果を取得する', 'parameters': {'type': 'object', 'properties': {'question': {'type': 'string', 'description': '質問文'}}, 'required': ['question']}}}]
------ask_question(言語モデルへの質問を行う関数)------
質問: 東京駅のイベントについて、最近1ヶ月以内の検索結果を教えてください
使用するツール: [{'type': 'function', 'function': {'name': 'get_search_result', 'description': '指定した質問文の検索結果を取得する', 'parameters': {'type': 'object', 'properties': {'question': {'type': 'string', 'description': '質問文'}}, 'required': ['question']}}}]
response.choices[0].finish_reasonの中身： tool_calls
------★★★★ツール呼び出し★★★★------
------handle_tool_call(ツール呼び出しが必要な場合の処理を行う関数)------
レスポンス: ChatCompletion(id='chatcmpl-C7ESrI351aRfO2mY2tOidlcKqnVWg', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, mes